1. 라이브러리 및 모듈

In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D
from keras.optimizers import SGD
import matplotlib.pyplot as plt

Using TensorFlow backend.


2. JSON 데이터 불러오기

In [2]:
import json

# JSON 파일 경로
json_path = r"C:\Users\juyeo\Desktop\틱택토_학습데이터\학습데이터\tictactoe_data_complete.json"

# JSON 데이터 불러오기
with open(json_path, 'r') as f:
    data = json.load(f)

# 내부 데이터 추출
game_data = data["game_data"]

# 특정 컬럼 10개씩 출력하는 함수
def print_column_samples(field_name, num_samples=10):
    print(f"\n--- {field_name} (Top {num_samples}) ---")
    for i in range(min(num_samples, len(game_data))):
        print(f"[{i}] {game_data[i][field_name]}")

# 출력
print_column_samples("board_state_before")
print_column_samples("board_state_current")
print_column_samples("action")
print_column_samples("player")
print_column_samples("winner")



--- board_state_before (Top 10) ---
[0] [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]
[1] [[0, 0, 0, 1], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]
[2] [[0, 0, 0, 1], [0, 0, 0, 0], [0, 0, 0, 0], [-1, 0, 0, 0]]
[3] [[0, 1, 0, 1], [0, 0, 0, 0], [0, 0, 0, 0], [-1, 0, 0, 0]]
[4] [[0, 1, 0, 1], [-1, 0, 0, 0], [0, 0, 0, 0], [-1, 0, 0, 0]]
[5] [[1, 1, 0, 1], [-1, 0, 0, 0], [0, 0, 0, 0], [-1, 0, 0, 0]]
[6] [[1, 1, 0, 1], [-1, -1, 0, 0], [0, 0, 0, 0], [-1, 0, 0, 0]]
[7] [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]
[8] [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1]]
[9] [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, -1, 1]]

--- board_state_current (Top 10) ---
[0] [[0, 0, 0, 1], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]
[1] [[0, 0, 0, 1], [0, 0, 0, 0], [0, 0, 0, 0], [-1, 0, 0, 0]]
[2] [[0, 1, 0, 1], [0, 0, 0, 0], [0, 0, 0, 0], [-1, 0, 0, 0]]
[3] [[0, 1, 0, 1], [-1, 0, 0, 0], [0, 0, 0, 0], [-1, 0, 0, 0]]
[4] [[1, 1, 0, 1], [-1, 0, 0, 0], [0, 0, 0, 0], [-1, 0, 0

3. 학습 데이터셋 x_train, y_train 준비

In [3]:
x_train = []
y_train = []

for item in game_data:
    board_state_before = item["board_state_before"]  # 길이 16짜리 1D 배열
    action = item["action"]  # 0~15 정수

    # 입력 데이터: (4,4,1)
    board_state_2d = np.array(board_state_before).reshape(4,4,1)
    x_train.append(board_state_2d)

    # 출력 데이터: (16,) one-hot 인코딩
    action_one_hot = np.zeros(16)
    action_one_hot[action] = 1
    y_train.append(action_one_hot)

x_train = np.array(x_train)
y_train = np.array(y_train)

print(f"x_train.shape: {x_train.shape}")
print(f"y_train.shape: {y_train.shape}")


x_train.shape: (12506, 4, 4, 1)
y_train.shape: (12506, 16)


4. Tic Tac Toe Environment 정의

In [4]:
class Environment():
    
    def __init__(self):
    # 보드는 0으로 초기화된 16개의 배열로 준비함
    # 게임종료 : done = True
        self.board_a = np.zeros(16)
        self.done = False
        self.reward = 0
        self.winner = 0
        self.print = False

    def move(self, p1, p2, player):
    # 각 플레이어가 선택한 행동을 표시 하고 게임 상태(진행 또는 종료)를 판단
    # p1 = 1, p2 = -1로 정의
    # 각 플레이어는 행동을 선택하는 select_action 메서드를 가짐
        if player == 1:
            pos = p1.select_action(self, player)
        else:
            pos = p2.select_action(self, player)
            
            
        # 보드에 플레이어의 선택을 표시
        self.board_a[pos] = player
        if self.print:
            print(player)
            self.print_board()
        # 게임이 종료상태인지 아닌지를 판단
        self.end_check(player)
        
        return self.reward, self.done
    
    # 현재 보드 상태에서 가능한 행동(둘 수 있는 장소)을 탐색하고 리스트로 반환
    def get_action(self):
        observation = []
        
        for i in range(16):
            if self.board_a[i] == 0:
                observation.append(i)
        return observation
    
    # 게임이 종료(승패 또는 비김)됐는지 판단
    def end_check(self, player):
        # 0   1   2   3
        # 4   5   6   7
        # 8   9  10  11
        #12  13  14  15
        
        # 4 x 4
        end_condition = [  
            # 가로
            (0,1,2,3), (4,5,6,7), (8,9,10,11), (12,13,14,15),
            # 세로
            (0,4,8,12), (1,5,9,13), (2,6,10,14), (3,7,11,15),
            # 대각선
            (0,5,10,15), (3,6,9,12)
        ]
        for line in end_condition:
             if self.board_a[line[0]] == self.board_a[line[1]] \
                and self.board_a[line[1]] == self.board_a[line[2]] \
                and self.board_a[line[2]] == self.board_a[line[3]] \
                and self.board_a[line[0]] != 0:  
                # 종료됐다면 누가 이겼는지 표시
                self.done = True
                self.reward = player
                return
        # 비긴 상태는 더는 보드에 빈 공간이 없을때
        observation = self.get_action()
        if (len(observation)) == 0:
            self.done = True
            self.reward = 0            
        return

    # 현재 보드의 상태를 표시 p1 = O, p2 = X    
    def print_board(self): #4x4로 수정 완료
        print("+----+----+----+----+")  
        for i in range(4): # 줄 수 
            for j in range(4):  # 칸 수 
                if self.board_a[4*i+j] == 1:  #index 계산식도 4*i+j로 바꿈
                    print("|  O",end=" ")
                elif self.board_a[4*i+j] == -1:
                    print("|  X",end=" ")
                else:
                    print("|   ",end=" ")
            print("|")
            print("+----+----+----+----+")  # 출력 테두리 4칸용으로 바꿈 

5. human player 정의

In [5]:
class Human_player():
    
    def __init__(self):
        self.name = "Human player"
        
    def select_action(self, env, player):
        while True:
            # 가능한 행동을 조사한 후 표시
            available_action = env.get_action()
            print("possible actions = {}".format(available_action))

            # 4x4 상태 번호 표시로 수정 완료
            print("+----+----+----+----+")  
            print("+  0 +  1 +  2 +  3 +") 
            print("+----+----+----+----+") 
            print("+  4 +  5 +  6 +  7 +")  
            print("+----+----+----+----+")  
            print("+  8 +  9 + 10 + 11 +")  
            print("+----+----+----+----+") 
            print("+ 12 + 13 + 14 + 15 +")  
            print("+----+----+----+----+")  
    
            # 키보드로 가능한 행동을 입력 받음
            action = input("Select action(human) : ")
            action = int(action)
            
            # 입력받은 행동이 가능한 행동이면 반복문을 탈출
            if action in available_action:
                return action
            # 아니면 행동 입력을 반복
            else:
                print("You selected wrong action")
        return

6. Random player 정의

In [6]:
class Random_player():
    def __init__(self):
        self.name = "Random player"

    def select_action(self, env, player):
        available_actions = env.get_action()
        return np.random.choice(available_actions)


7. CNN player 정의

In [7]:
class CNN_player():
    def __init__(self):
        self.name = "CNN_player"
        self.model = self.build_model()
        self.print = False

    def build_model(self):
        
        # CNN 모델 생성
        model = Sequential()
        
        # 합성곱층: 4x4 입력, 채널=1, 필터=32, relu
        model.add(Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(4,4,1)))
        model.add(Flatten())
        
        # 완전연결층
        model.add(Dense(64, activation='relu'))
        model.add(Dense(16, activation='linear')) # 16개의 출력: 각 칸의 점수
        model.compile(optimizer=SGD(lr=0.01), loss='mean_squared_error', metrics=['mse'])
        #원래 학습률 : 0.01
        
        return model

    def state_convert(self, board_a):
        
         # 1D 배열(16칸)을 4x4x1 배열로 변환
        state = np.zeros((4,4,1))
        
        for i in range(16):
            state[i//4, i%4, 0] = board_a[i]
            
        return np.expand_dims(state, axis=0)

    def select_action(self, env, player):
        
        # CNN 모델로 Q-value 예측 → argmax 선택
        state = self.state_convert(env.board_a)
        q_values = self.model.predict(state, verbose=0)[0]
        available_actions = env.get_action()
        available_q_values = q_values[available_actions]
        selected_index = np.argmax(available_q_values)
        action = available_actions[selected_index]
        
        if self.print:
            print(f"Q-values: {np.round(q_values,2)}")
            print(f"Available actions: {available_actions}")
            print(f"Selected action: {action}")
        return action

    def train(self, x_train, y_train, epochs=20, verbose=1):
        self.model.fit(x_train, y_train, epochs=epochs, verbose=verbose)


8. CNN 모델 학습

In [8]:
cnn_player = CNN_player()

print("CNN 모델 학습 시작...")
cnn_player.train(x_train, y_train, epochs=30, verbose=1) #원래 에폭 30
print("학습 완료!")





CNN 모델 학습 시작...



Epoch 1/30





12506/12506 [==============================] - 1s 85us/step - loss: 0.0657 - mean_squared_error: 0.0657
Epoch 2/30
12506/12506 [==============================] - 1s 65us/step - loss: 0.0620 - mean_squared_error: 0.0620
Epoch 3/30
12506/12506 [==============================] - 1s 65us/step - loss: 0.0609 - mean_squared_error: 0.0609
Epoch 4/30
12506/12506 [==============================] - 1s 85us/step - loss: 0.0604 - mean_squared_error: 0.0604
Epoch 5/30
12506/12506 [==============================] - 1s 89us/step - loss: 0.0600 - mean_squared_error: 0.0600
Epoch 6/30
12506/12506 [==============================] - 1s 91us/step - loss: 0.0598 - mean_squared_error: 0.0598
Epoch 7/30
12506/12506 [==============================] - 1s 87us/step - loss: 0.0596 - mean_squared_error: 0.0596
Epoch 8/30
12506/12506 [==============================] - 1s 88us/step - loss: 0.0595 - mean_squared_error: 0.0595
Epoch 9/30
12506/12506 [============================

9. 게임 실행 함수

In [9]:
## np.random.seed(0)

# p1 = cnn_player
# p2 = Human_player()

p1 = Human_player()
p2 = cnn_player

# p1 = trained_p1
# p2 = trained_p2
# p2 = Human_player()

# 지정된 게임 수를 자동으로 두게 할 것인지 한게임씩 두게 할 것인지 결정
# auto = True : 지정된 판수(games)를 자동으로 진행 
# auto = False : 한판씩 진행

auto = False

# auto 모드의 게임수
games = 100

print("pl player : {}".format(p1.name))
print("p2 player : {}".format(p2.name))

# 각 플레이어의 승리 횟수를 저장
p1_score = 0
p2_score = 0
draw_score = 0


if auto: 
    # 자동 모드 실행
    for j in tqdm(range(games)):
        
        np.random.seed(j)
        env = Environment()
        
        for i in range(10000):
            # p1 과 p2가 번갈아 가면서 게임을 진행
            # p1(1) -> p2(-1) -> p1(1) -> p2(-1) ...
            reward, done = env.move(p1,p2,(-1)**i)
            # 게임 종료 체크
            if done == True:
                if reward == 1:
                    p1_score += 1
                elif reward == -1:
                    p2_score += 1
                else:
                    draw_score += 1
                break

else:                
    # 한 게임씩 진행하는 수동 모드
    np.random.seed(1)
    while True:
        
        env = Environment()
        env.print = False
        for i in range(10000):
            reward, done = env.move(p1,p2,(-1)**i)
            env.print_board()
            if done == True:
                if reward == 1:
                    print("winner is p1({})".format(p1.name))
                    p1_score += 1
                elif reward == -1:
                    print("winner is p2({})".format(p2.name))
                    p2_score += 1
                else:
                    print("draw")
                    draw_score += 1
                break
        
        # 최종 결과 출력        
        print("final result")
        env.print_board()

        # 한게임 더?최종 결과 출력 
        answer = input("More Game? (y/n)")

        if answer == 'n':
            break           

print("p1({}) = {} p2({}) = {} draw = {}".format(p1.name, p1_score,p2.name, p2_score,draw_score))
                

pl player : Human player
p2 player : CNN_player
possible actions = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
+----+----+----+----+
+  0 +  1 +  2 +  3 +
+----+----+----+----+
+  4 +  5 +  6 +  7 +
+----+----+----+----+
+  8 +  9 + 10 + 11 +
+----+----+----+----+
+ 12 + 13 + 14 + 15 +
+----+----+----+----+
Select action(human) : 15
+----+----+----+----+
|    |    |    |    |
+----+----+----+----+
|    |    |    |    |
+----+----+----+----+
|    |    |    |    |
+----+----+----+----+
|    |    |    |  O |
+----+----+----+----+
+----+----+----+----+
|    |    |    |  X |
+----+----+----+----+
|    |    |    |    |
+----+----+----+----+
|    |    |    |    |
+----+----+----+----+
|    |    |    |  O |
+----+----+----+----+
possible actions = [0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
+----+----+----+----+
+  0 +  1 +  2 +  3 +
+----+----+----+----+
+  4 +  5 +  6 +  7 +
+----+----+----+----+
+  8 +  9 + 10 + 11 +
+----+----+----+----+
+ 12 + 13 + 14 + 15 +
+----+----+----